# Sistem Rekomendasi Film Menggunakan Content-Based dan Collaborative Filtering

## Project Overview
Sistem rekomendasi film bertujuan membantu pengguna menemukan film yang sesuai dengan preferensi mereka secara otomatis. Dengan banyaknya pilihan film, sistem ini dapat meningkatkan pengalaman pengguna dan membantu platform streaming meningkatkan engagement.

## Business Understanding
Permasalahan: Pengguna sering kesulitan memilih film yang sesuai dengan selera mereka di antara ribuan pilihan.
Solusi: Membangun sistem rekomendasi yang dapat memberikan saran film secara personal menggunakan dua pendekatan utama:
- **Content-Based Filtering**: Merekomendasikan film berdasarkan kemiripan konten (genre, deskripsi, dsb.).
- **Collaborative Filtering**: Merekomendasikan film berdasarkan pola rating pengguna lain yang mirip.

In [ ]:
# Data Understanding
import pandas as pd

# Baca file movies.dat
movies = pd.read_csv(
    'datasets/movies.dat',
    sep='::',
    engine='python',
    names=['MovieID', 'Title', 'Genres']
)

# Baca file ratings.dat
ratings = pd.read_csv(
    'datasets/ratings.dat',
    sep='::',
    engine='python',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp']
)

# Tampilkan beberapa data awal
print("Contoh data film:")
display(movies.head())

print("Contoh data rating:")
display(ratings.head())